## Geneformer Embeddings

### Imports

In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
from env.imports import *

In [27]:
import importlib
import data

import models
import sim.sim
import sim.sim_utils
from sim.sim_utils import bytes2human, print_system_usage
from sim.sim import Simulation
from sim.sim_run import single_sim_run

importlib.reload(sim.sim)
importlib.reload(sim.sim_run)
importlib.reload(models.train_val)
importlib.reload(data.data_utils)


<module 'data.data_utils' from '/scratch/asr655/neuroinformatics/GeneEx2Conn/data/data_utils.py'>

#### Check job specs

In [28]:
print_system_usage()

total = psutil.disk_usage('/').total
print(bytes2human(total))

CPU Usage: 2.7%
RAM Usage: 5.4%
Available RAM: 952.3G
Total RAM: 1007.0G
52.4G


In [29]:
print(os.environ.get("CUDA_VISIBLE_DEVICES"))

# Check available GPUs
print(f"Available GPUs: {torch.cuda.device_count()}")

for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)} - Memory Allocated: {torch.cuda.memory_allocated(i)/1024**3:.2f} GB")


None
Available GPUs: 1
GPU 0: NVIDIA A100-SXM4-80GB - Memory Allocated: 0.00 GB


### Setup

In [34]:
from data.data_load import load_transcriptome, load_connectome, load_coords, load_network_labels
from data.data_viz import plot_connectome, plot_transcriptome
from data.data_utils import expand_X_symmetric, expand_Y_symmetric
from data.data_utils import RegionPairDataset
from torch.utils.data import Subset

import anndata
from anndata import AnnData

from geneformer import TranscriptomeTokenizer
from geneformer import EmbExtractor
import torch

from torch.utils.data import TensorDataset, DataLoader


In [35]:
import pandas as pd
import numpy as np
import anndata as ad
import os
from geneformer import TranscriptomeTokenizer

# === STEP 1: Load preprocessed matrix ===
X_filtered = pd.read_csv("./data/geneformer/filtered_expression.csv", index_col=0)  # shape: (455, 14111)
ensembl_ids = list(X_filtered.columns)

# === STEP 2: Construct AnnData object ===
adata = ad.AnnData(X_filtered.values)
adata.obs_names = X_filtered.index
adata.var_names = ensembl_ids
adata.var["ensembl_id"] = adata.var_names

# --- drop ensembl_id_collapsed if present ---
if "ensembl_id_collapsed" in adata.var.columns:
    adata.var.drop(columns=["ensembl_id_collapsed"], inplace=True)

# Add n_counts column
adata.obs["n_counts"] = X_filtered.sum(axis=1).values
# Add custom attribute
adata.obs["organ_major"] = "brain"  # you can replace this with actual metadata per region

# === STEP 3: Save to .h5ad file ===
save_path = "./data/geneformer/ahba_pseudocounts_clean.h5ad"
adata.write_h5ad(save_path)
print(f"✅ Saved cleaned AnnData: {save_path}")

FileNotFoundError: [Errno 2] No such file or directory: './data/geneformer/filtered_expression.csv'